# Text Classification Feature Generation

Just found some methods to generate text features in a way I haven't tried before, looks like something worthy to try. It's also interesting to add some sentiment features.


* Reference: https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/?utm_source=facebook.com
* Data source: https://drive.google.com/drive/folders/0Bz8a_Dbh9Qhbfll6bVpmNUtUcFdjYmF2SEpmZUZUcVNiMUw1TWN6RDV3a0JHT3kxLVhVR2M

In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

## About Raw Data

In [7]:
data_folder = 'yelp_review_full_csv/'
train = pd.read_csv(data_folder + 'train.csv', header=None)
test = pd.read_csv(data_folder + 'test.csv', header=None)
train.columns = ['class', 'yelp_review']
test.columns = ['class', 'yelp_review']

print(train.shape, test.shape)
train.head()

((650000, 2), (50000, 2))


,class,yelp_review
0,5,dr. goldberg offers everything i look for in a...
1,2,"Unfortunately, the frustration of being Dr. Go..."
2,4,Been going to Dr. Goldberg for over 10 years. ...
3,4,Got a letter in the mail last week that said D...
4,1,I don't know what Dr. Goldberg was like before...


In [9]:
# what a balanced dataset... 5 is the best review
print(train['class'].value_counts())
print(test['class'].value_counts())

5    130000
4    130000
3    130000
2    130000
1    130000
Name: class, dtype: int64
5    10000
4    10000
3    10000
2    10000
1    10000
Name: class, dtype: int64


In [12]:
# look at the length of the reviews...! How did they generated these data...?
train['review_len'] = len(train['yelp_review'])
print(min(train['review_len']), max(train['review_len']))

test['review_len'] = len(test['yelp_review'])
print(min(test['review_len']), max(test['review_len']))

(650000, 650000)
(50000, 50000)


## Text Feature Generation